# Analyse et Système de Recommandation de Chansons

Ce notebook reprend et adapte le script `recommendation_system.py` pour une exécution interactive. Il compare deux approches de vectorisation de texte (TF-IDF et Embeddings) pour un système de recommandation de chansons.

## Section 0: Imports et Configuration

In [ ]:
import os
import glob
import re
import string
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# --- Configuration initiale de NLTK ---
print('--- Cellule 0: Imports et Configuration NLTK ---')
try:
    nltk.data.find('corpora/stopwords')
    print('Stopwords NLTK déjà téléchargés.')
except LookupError:
    print('Téléchargement des stopwords NLTK...')
    nltk.download('stopwords')

stop_words_fr = set(stopwords.words('french'))
print('Librairies importées et stopwords configurés.')

--- Cellule 0: Imports et Configuration NLTK ---
Stopwords NLTK déjà téléchargés.
Librairies importées et stopwords configurés.


## Section 1: Chargement et Prétraitement des Données

In [2]:
print('\n--- Cellule 1: Définition de la fonction de nettoyage ---')
def clean_text(text):
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation])
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    words = text.split()
    words = [word for word in words if word not in stop_words_fr]
    return ' '.join(words)

print('Fonction clean_text définie.')


print('\n--- Cellule 1.1: Chargement et nettoyage du corpus ---')
# Le chemin vers le dossier 'TP2/chansons' est supposé être relatif au dossier où le notebook est exécuté
chansons_directory = os.path.join(os.getcwd(), 'TP2', 'chansons')

print(f"Chargement des données depuis : {chansons_directory}")
filepaths = glob.glob(os.path.join(chansons_directory, '*.txt'))

if not filepaths:
    print(f"ERREUR: Aucun fichier .txt n'a été trouvé dans le dossier '{chansons_directory}'.")
    print("Veuillez vérifier que le chemin est correct et que les fichiers existent.")
    df_chansons = pd.DataFrame()
else:
    chansons_data = []
    for filepath in filepaths:
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                lyrics = f.read()
            title = os.path.basename(filepath).replace('.txt', '').replace('_', ' ').title()
            chansons_data.append({'titre': title, 'paroles': lyrics})
        except Exception as e:
            print(f"Erreur de lecture du fichier {filepath}: {e}")

    df_chansons = pd.DataFrame(chansons_data)
    print(f"{len(df_chansons)} chansons chargées.")
    
    print('Nettoyage des paroles en cours...')
    df_chansons['paroles_cleaned'] = df_chansons['paroles'].apply(clean_text)
    
    print('Affichage des 5 premières lignes du DataFrame:')
    display(df_chansons.head())


--- Cellule 1: Définition de la fonction de nettoyage ---
Fonction clean_text définie.

--- Cellule 1.1: Chargement et nettoyage du corpus ---
Chargement des données depuis : /content/TP2/chansons
ERREUR: Aucun fichier .txt n'a été trouvé dans le dossier '/content/TP2/chansons'.
Veuillez vérifier que le chemin est correct et que les fichiers existent.


## Section 2: Vectorisation (Comparaison des deux approches)

### Approche 1: TF-IDF (Classique)

In [ ]:
print('\n--- Cellule 2.1: Vectorisation avec TF-IDF ---')
def vectorize_tfidf(texts):
    vectorizer = TfidfVectorizer()
    matrix = vectorizer.fit_transform(texts)
    print(f"Matrice TF-IDF créée. Dimensions : {matrix.shape}")
    return matrix

if not df_chansons.empty:
    tfidf_matrix = vectorize_tfidf(df_chansons['paroles_cleaned'])
    cosine_sim_tfidf = cosine_similarity(tfidf_matrix)
else:
    tfidf_matrix = None
    cosine_sim_tfidf = None
    print('Le DataFrame est vide, la vectorisation TF-IDF est annulée.')

### Approche 2: Embeddings Sémantiques (Moderne)

In [ ]:
print('\n--- Cellule 2.2: Vectorisation avec Embeddings Sémantiques ---')
def vectorize_embeddings(texts):
    try:
        from sentence_transformers import SentenceTransformer
    except ImportError:
        print("ERREUR: La librairie 'sentence-transformers' n'est pas installée.")
        print("Veuillez l'installer avec la commande : pip install sentence-transformers")
        return None

    model = SentenceTransformer('distiluse-base-multilingual-cased-v1')
    print("Modèle d'embedding chargé. Création des embeddings en cours...")
    embeddings = model.encode(texts.tolist(), show_progress_bar=True)
    print(f"Matrice d'embeddings créée. Dimensions : {embeddings.shape}")
    return embeddings

if not df_chansons.empty:
    embedding_matrix = vectorize_embeddings(df_chansons['paroles_cleaned'])
    if embedding_matrix is not None:
        cosine_sim_embed = cosine_similarity(embedding_matrix)
    else:
        cosine_sim_embed = None
else:
    embedding_matrix = None
    cosine_sim_embed = None
    print('Le DataFrame est vide, la vectorisation par embeddings est annulée.')

## Section 3: Logique de Recommandation

In [ ]:
print('\n--- Cellule 3.1: Définition de la fonction de recommandation ---')
def get_recommendations(song_title, df, similarity_matrix, top_n=5):
    if song_title not in df['titre'].values:
        print(f"ERREUR: Le titre '{song_title}' n'est pas dans la liste des chansons.")
        return

    idx = df.index[df['titre'] == song_title].tolist()[0]
    sim_scores = list(enumerate(similarity_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_song_indices = [i[0] for i in sim_scores[1:top_n+1]]
    
    recommendations = df['titre'].iloc[top_song_indices].tolist()
    return recommendations


if not df_chansons.empty:
    # Choisir une chanson pour tester les recommandations (ici, la première du DataFrame)
    test_song_title = df_chansons['titre'].iloc[0]
    print(f"\n--- Cellule 3.2: Recommandations pour '{test_song_title}' ---")

    # --- Recommandations avec TF-IDF ---
    if cosine_sim_tfidf is not None:
        print("\n*** RÉSULTATS AVEC TF-IDF ***")
        tfidf_recs = get_recommendations(test_song_title, df_chansons, cosine_sim_tfidf)
        for i, rec_title in enumerate(tfidf_recs):
            print(f"{i+1}. {rec_title}")

    # --- Recommandations avec Embeddings ---
    if cosine_sim_embed is not None:
        print("\n*** RÉSULTATS AVEC EMBEDDINGS (APPROCHE MODERNE) ***")
        embed_recs = get_recommendations(test_song_title, df_chansons, cosine_sim_embed)
        for i, rec_title in enumerate(embed_recs):
            print(f"{i+1}. {rec_title}")
else:
    print('Le DataFrame est vide, les recommandations ne peuvent pas être générées.')

## Section 4: Visualisation des Embeddings

In [ ]:
print('\n--- Cellule 4.1: Visualisation des Embeddings avec PCA ---')
def visualize_embeddings_pca(embedding_matrix, df):
    if embedding_matrix is None or df.empty:
        print("Matrice d'embeddings ou DataFrame vide, impossible de visualiser.")
        return

    pca = PCA(n_components=2)
    embeddings_2d = pca.fit_transform(embedding_matrix)
    
    plt.figure(figsize=(14, 10))
    plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], alpha=0.7)
    
    # Annoter quelques points pour la lisibilité
    # S'assure que len(df) n'est pas 0 avant la division
    annotation_interval = max(1, len(df) // 10) # Annoter environ 10% des chansons
    for i, title in enumerate(df['titre']):
        if i % annotation_interval == 0:
             plt.annotate(title, (embeddings_2d[i, 0], embeddings_2d[i, 1]), alpha=0.8)
    
    plt.title("Visualisation des Chansons (Embeddings projetés en 2D via PCA)")
    plt.xlabel("Composante Principale 1")
    plt.ylabel("Composante Principale 2")
    plt.grid(True)
    
    output_filename = "visualisation_chansons_pca_notebook.png"
    plt.savefig(output_filename)
    print(f"Le graphique de visualisation a été sauvegardé sous : {output_filename}")
    plt.show() # Pour afficher le graphique directement dans le notebook


if not df_chansons.empty and embedding_matrix is not None:
    visualize_embeddings_pca(embedding_matrix, df_chansons)
else:
    print('Visualisation annulée car le DataFrame ou la matrice d\'embeddings est vide.')